In [9]:
import duckdb
import pickle
import pandas as pd

In [6]:
with open('experiments/results/meta.pkl', 'rb') as f:
    vocab, words_member = pickle.load(f)

In [12]:
# Write
dfm = pd.DataFrame(words_member, columns=['word', 'member'])
dfm['index'] = dfm.index
dfm['filename'] = dfm.word.apply(lambda x: f'cc-binaries/{x}')
dfm.to_parquet('sfdata/words_member.parquet')

In [31]:
res = duckdb.sql("""
    select distinct (*) from read_parquet('experiments/results/parquet/df*.parquet', union_by_name=true)
    limit 10
""")
res.show()

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬──────────────────────────────────────────────────┬───────┬───────┬───────┬───────┐
│ index │   i   │   0   │   1   │   2   │   3   │   4   │   5   │   6   │   7   │   8   │   9   │  

In [41]:
res = duckdb.sql("""
    with sheets as (
        select * from read_parquet('experiments/results/parquet/df*.parquet', union_by_name=true)
    ), triples as (
        UNPIVOT sheets
        ON COLUMNS(* EXCLUDE (index, fs, f, s, i, rows, cols))
        INTO
            NAME j
            VALUE v
    ), pairs as (
        select distinct
            s1.fs as fs,
            s1.v as v1,
            s2.v as v2,
        from triples s1
        join triples s2 on s1.j = s2.j and s1.i <= s2.i and s1.fs = s2.fs
    )
    select p1.fs, p2.fs, p1.v1, p1.v2
    from pairs p1 join pairs p2 on p1.v1 = p2.v1 and p1.v2 = p2.v2 and p1.fs < p2.fs
    limit 10
""")
res

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬───────┬───────┬───────┐
│  fs   │  fs   │  v1   │  v2   │
│ int64 │ int64 │ int64 │ int64 │
├───────┼───────┼───────┼───────┤
│    94 │    94 │  4886 │  4751 │
│    94 │    94 │  4935 │  4637 │
│    94 │    94 │  5000 │  4418 │
│    94 │    94 │  5074 │  4688 │
│    94 │    94 │  4819 │  4843 │
│    94 │    94 │  5044 │  4870 │
│    94 │    94 │  5044 │  4800 │
│    94 │    94 │  4819 │  4622 │
│    94 │    94 │  4819 │  4719 │
│    94 │    94 │  5044 │  4491 │
├───────┴───────┴───────┴───────┤
│ 10 rows             4 columns │
└───────────────────────────────┘